In [ ]:
import pandas as pd

"""
Calculate your teaching hours.
"""

# NOTE missing a check if a long session goes over lunch and remove an hour
# NOTE unclear if things like course development and presentations get a first time multiplier too
# NOTE at the moment, the academic hour (15 minutes in the beginning) is granted only if its 15 minutes past the hour

# load data (ONLY ENGLISH SUPPORTED, you can select the language in TimeEdit)
data = pd.read_csv("data/TimeEdit_Ezra_Eisbrenner_2024-12-21_14_43.csv", header=2)
data.head(3)

,Begin date,Begin time,End date,End time,Course,Module,Course component,Activities,Student group,Teacher,Unnamed: 10,Info,Required reading,Equipment,Important message,URL,"Room,Building,Floor",Map,Access #,Equipment OOS
0,2024-09-06,15:00,2024-09-06,17:00,NaN,NaN,NaN,Other,NaN,NaN,MISU,NaN,NaN,MISU Intern,NaN,NaN,MISU Pentry. Arrhenius hus C. Vån 6,https://www.openstreetmap.org/?mlat=59.36560&m...,NaN,NaN
1,2024-09-19,16:00,2024-09-19,18:00,NaN,NaN,NaN,Doktorandråd,NaN,NaN,PhD students,NaN,NaN,MISU Intern,NaN,NaN,C502a Kuling. Arrhenius hus C. Vån 5,https://www.openstreetmap.org/?mlat=59.36560&m...,NaN,NaN
2,2024-10-03,13:15,2024-10-03,14:00,MO8004. Modelling of Large-Scale Circulation i...,NaN,NaN,Computer lab.,NaN,Ezra Eisbrenner,NaN,NaN,NaN,NaN,NaN,NaN,C508 Ekman. Arrhenius hus C. Vån 5,https://www.openstreetmap.org/?mlat=59.36560&m...,NaN,NaN


In [ ]:
# define courses
largescale = data.Course.unique()[1]
mesoscale = data.Course.unique()[2]

# Graded reports is the number of reports you have to grade if there was 1 student.
# Time for grading does not scale with number of students,
# as per decision by the department. You don't get hours for pass-fail evaluation.
courses = {
    largescale: {"first_time": False, "n_graded_reports": 0.5},
    mesoscale: {"first_time": False, "n_graded_reports": 0},
}

In [23]:
# ==================================================================================== #
# No change needed below here!
# Unless code is broken.
# ==================================================================================== #

In [24]:
factors = {
    "Computer lab.": 2,
    "Laboratory experiment": 2,
    "Presentation": 1,
    "Course development": 1,
    "Calculation exercise": 4,
}
first_time_multiplier = 1.5

for t in ["Begin", "End"]:
    data[t] = data.apply(
        lambda r: pd.Timestamp(" ".join([r[f"{t} date"], r[f"{t} time"]])), axis=1
    )
data["Begin"] = data["Begin"].apply(
    lambda x: x - pd.Timedelta(minutes=15 if x.minute == 15 else 0)
)
data["duration"] = data["End"] - data["Begin"]

for course in courses:
    course_data = data.loc[lambda r: r.Course == course]

    break_later = False
    for activity in course_data.Activities:
        if activity not in factors.keys():
            print(f"{activity} has no assigned factor.")
            break_later = True
    if break_later:
        raise KeyError("Code breaks because there is an unknown activity.")

    totaltime = 0
    print(f"{course}\n")
    for activity, factor in factors.items():
        time = course_data.loc[
            lambda r: r.Activities == activity
        ].duration.sum().total_seconds() / (60 * 60)
        print(f"{activity}: {time} x {factor}")
        totaltime = totaltime + time * factor

    print(f'Report grading: {float(courses[course]["n_graded_reports"])} x 4')
    totaltime = totaltime + courses[course]["n_graded_reports"] * 4

    if courses[course]["first_time"]:
        totaltime = totaltime * first_time_multiplier
    print(f'First time: {courses[course]["first_time"]}')
    print(f"Time in total: {totaltime} hours\n")

MO8004. Modelling of Large-Scale Circulation in Atmosphere and Ocean

Computer lab.: 31.0 x 2
Laboratory experiment: 0.0 x 2
Presentation: 5.0 x 1
Course development: 0.0 x 1
Calculation exercise: 4.0 x 4
Report grading: 0.5 x 4
First time: False
Time in total: 85.0 hours

MO8005. Mesoscale Meteorology

Computer lab.: 9.0 x 2
Laboratory experiment: 0.0 x 2
Presentation: 0.0 x 1
Course development: 0.0 x 1
Calculation exercise: 7.75 x 4
Report grading: 0.0 x 4
First time: False
Time in total: 49.0 hours

